In [1]:
# YOLACT-Edge Colab Training and Export Pipeline for Custom Dataset

from google.colab import drive

drive.mount('/content/drive')  # Optional, if dataset/model is in Drive



Mounted at /content/drive


In [2]:
!unzip "/content/drive/MyDrive/datasets/strawberry-seg-synthetic.zip" -d "/content"

Streaming output truncated to the last 5000 lines.
  inflating: /content/Strawberry-segmentation-synthetic/valid/aug_56_gray_mold68_jpg.rf.e110bd364819a2c3117baecc89d77b9e.jpg  
  inflating: /content/Strawberry-segmentation-synthetic/valid/powdery_mildew_leaf188_jpg.rf.575cd43449d053d3922590e89fb973ae.jpg  
  inflating: /content/Strawberry-segmentation-synthetic/valid/aug_407_anthracnose_fruit_rot23_jpg.rf.e78db260f6bc10256856b89c5a04155a.jpg  
  inflating: /content/Strawberry-segmentation-synthetic/valid/aug_39_powdery_mildew_leaf59_jpg.rf.1c8f8cd117a64044e0d1c18d69eb0735.jpg  
  inflating: /content/Strawberry-segmentation-synthetic/valid/powdery_mildew_fruit53_jpg.rf.860644a1c0ccc4c9da3541000cb81a11.jpg  
  inflating: /content/Strawberry-segmentation-synthetic/valid/leaf_spot346_jpg.rf.761b85371a7372f9b4236f2d4d2b689d.jpg  
  inflating: /content/Strawberry-segmentation-synthetic/valid/aug_296_blossom_blight14_jpg.rf.6ffe0f70914b13c073b0385b106f13f7.jpg  
  inflating: /content/Strawbe

In [3]:
# 1. Clone the Repository
!git clone https://github.com/edramos-lab/yolact_edge.git
%cd yolact_edge

# 2. Install Required Packages

!pip install pycocotools opencv-python
!apt-get install ninja-build
!pip install git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI
!pip install pycocotools opencv-python pillow


# 3. Set Up Dataset (COCO Format Expected)
# Expected structure:
# dataset/
# ├── images/
# │   ├── train2017/*.jpg
# │   └── val2017/*.jpg
# └── annotations/
#     ├── instances_train2017.json
#     └── instances_val2017.json

# 4. Modify Config for Custom Dataset (Optional: Use sed or do manually)
# If using a custom config file:
# !cp config/yolact_edge_mobilenetv2_config.py config/yolact_edge_custom_config.py


Cloning into 'yolact_edge'...
remote: Enumerating objects: 471, done.
remote: Counting objects: 100% (177/177), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 471 (delta 106), reused 74 (delta 74), pack-reused 294 (from 2)
Receiving objects: 100% (471/471), 24.12 MiB | 31.50 MiB/s, done.
Resolving deltas: 100% (254/254), done.
/content/yolact_edge
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  ninja-build
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 111 kB of archives.
After this operation, 358 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 ninja-build amd64 1.10.1-1 [111 kB]
Fetched 111 kB in 1s (185 kB/s)
Selecting previously unselected package ninja-build.
(Reading database ... 126281 files and directories currently installed.)
Preparing to unpack .../ninja-build_1.10.1-1_amd64.

In [7]:
!cd /content/yolact_edge/

In [14]:
# Copy this entire cell and paste it into your Colab notebook after cloning the repository:

import os
import subprocess
import sys

def install_cython():
    """Install Cython if not already installed."""
    try:
        import cython
        print("✅ Cython already installed")
    except ImportError:
        print("📦 Installing Cython...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "cython"])
        print("✅ Cython installed successfully")

def ultimate_fix_yolact_edge():
    """Ultimate comprehensive fix for all yolact_edge issues."""
    
    print("🔧 Applying ultimate comprehensive fixes to yolact_edge...")
    
    # Step 1: Install Cython
    install_cython()
    
    # Fix 1: Completely rewrite the Cython file
    cython_file = "yolact_edge/utils/cython_nms.pyx"
    if os.path.exists(cython_file):
        fixed_cython_content = '''# cython: language_level=3
## Note: Figure out the license details later.
#
# Based on:
# --------------------------------------------------------
# Fast R-CNN
# Copyright (c) 2015 Microsoft
# Licensed under The MIT License [see LICENSE for details]
# Written by Ross Girshick
# --------------------------------------------------------

cimport cython
import numpy as np
cimport numpy as np

cdef inline np.float32_t max(np.float32_t a, np.float32_t b) nogil:
    return a if a >= b else b

cdef inline np.float32_t min(np.float32_t a, np.float32_t b) nogil:
    return a if a <= b else b

@cython.boundscheck(False)
@cython.cdivision(True)
@cython.wraparound(False)
def nms(np.ndarray[np.float32_t, ndim=2] dets, np.float32_t thresh):
    cdef np.ndarray[np.float32_t, ndim=1] x1 = dets[:, 0]
    cdef np.ndarray[np.float32_t, ndim=1] y1 = dets[:, 1]
    cdef np.ndarray[np.float32_t, ndim=1] x2 = dets[:, 2]
    cdef np.ndarray[np.float32_t, ndim=1] y2 = dets[:, 3]
    cdef np.ndarray[np.float32_t, ndim=1] scores = dets[:, 4]

    cdef np.ndarray[np.float32_t, ndim=1] areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    cdef np.ndarray[np.int64_t, ndim=1] order = scores.argsort()[::-1]

    cdef int ndets = dets.shape[0]
    cdef np.ndarray[np.int32_t, ndim=1] suppressed = \\
            np.zeros((ndets), dtype=np.int32)

    # nominal indices
    cdef int _i, _j
    # sorted indices
    cdef int i, j
    # temp variables for box i's (the box currently under consideration)
    cdef np.float32_t ix1, iy1, ix2, iy2, iarea
    # variables for computing overlap with box j (lower scoring box)
    cdef np.float32_t xx1, yy1, xx2, yy2
    cdef np.float32_t w, h
    cdef np.float32_t inter, ovr

    with nogil:
      for _i in range(ndets):
          i = order[_i]
          if suppressed[i] == 1:
              continue
          ix1 = x1[i]
          iy1 = y1[i]
          ix2 = x2[i]
          iy2 = y2[i]
          iarea = areas[i]
          for _j in range(_i + 1, ndets):
              j = order[_j]
              if suppressed[j] == 1:
                  continue
              xx1 = max(ix1, x1[j])
              yy1 = max(iy1, y1[j])
              xx2 = min(ix2, x2[j])
              yy2 = min(iy2, y2[j])
              w = max(0.0, xx2 - xx1 + 1)
              h = max(0.0, yy2 - yy1 + 1)
              inter = w * h
              ovr = inter / (iarea + areas[j] - inter)
              if ovr >= thresh:
                  suppressed[j] = 1

    return np.where(suppressed == 0)[0]
'''
        
        with open(cython_file, 'w') as f:
            f.write(fixed_cython_content)
        print("✅ Cython file completely rewritten")
    
    # Fix 2: Fix the import in config.py
    config_file = "yolact_edge/data/config.py"
    if os.path.exists(config_file):
        with open(config_file, 'r') as f:
            content = f.read()
        
        # Replace the import line
        old_import = "from backbone import ResNetBackbone, VGGBackbone, ResNetBackboneGN, DarkNetBackbone, MobileNetV2Backbone"
        new_import = "from yolact_edge.backbone import ResNetBackbone, VGGBackbone, ResNetBackboneGN, DarkNetBackbone, MobileNetV2Backbone"
        
        if old_import in content:
            content = content.replace(old_import, new_import)
            print("✅ Fixed import in config.py")
        else:
            print("⚠️ Import line not found or already fixed")
        
        with open(config_file, 'w') as f:
            f.write(content)
    
    # Fix 3: Fix the CUDA issue in yolact.py
    yolact_file = "yolact_edge/yolact.py"
    if os.path.exists(yolact_file):
        with open(yolact_file, 'r') as f:
            content = f.read()
        
        # Find and replace the problematic CUDA call
        old_line = "torch.cuda.current_device()"
        new_line = "# torch.cuda.current_device()  # Commented out for CPU-only environments"
        
        if old_line in content:
            content = content.replace(old_line, new_line)
            print("✅ Fixed CUDA issue in yolact.py")
        else:
            print("⚠️ CUDA line not found or already fixed")
        
        with open(yolact_file, 'w') as f:
            f.write(content)
    
    # Fix 4: Fix the min_size configuration issue in augmentations.py
    augmentations_file = "yolact_edge/utils/augmentations.py"
    if os.path.exists(augmentations_file):
        with open(augmentations_file, 'r') as f:
            content = f.read()
        
        # Replace the min_size assignment to use getattr for backward compatibility
        old_line = "        self.min_size = cfg.min_size"
        new_line = "        # Use max_size as min_size if min_size is not defined (for backward compatibility)\n        self.min_size = getattr(cfg, 'min_size', cfg.max_size)"
        
        if old_line in content:
            content = content.replace(old_line, new_line)
            print("✅ Fixed min_size configuration issue in augmentations.py")
        else:
            print("⚠️ min_size line not found or already fixed")
        
        with open(augmentations_file, 'w') as f:
            f.write(content)
    
    # Fix 5: Fix the min_size configuration issue in output_utils.py
    output_utils_file = "yolact_edge/layers/output_utils.py"
    if os.path.exists(output_utils_file):
        with open(output_utils_file, 'r') as f:
            content = f.read()
        
        # Replace both instances of cfg.min_size with getattr
        old_line1 = "        r_w, r_h = Resize.faster_rcnn_scale(w, h, cfg.min_size, cfg.max_size)"
        new_line1 = "        # Use max_size as min_size if min_size is not defined (for backward compatibility)\n        min_size = getattr(cfg, 'min_size', cfg.max_size)\n        r_w, r_h = Resize.faster_rcnn_scale(w, h, min_size, cfg.max_size)"
        
        old_line2 = "        r_w, r_h = Resize.faster_rcnn_scale(w, h, cfg.min_size, cfg.max_size)"
        new_line2 = "        # Use max_size as min_size if min_size is not defined (for backward compatibility)\n        min_size = getattr(cfg, 'min_size', cfg.max_size)\n        r_w, r_h = Resize.faster_rcnn_scale(w, h, min_size, cfg.max_size)"
        
        if old_line1 in content:
            content = content.replace(old_line1, new_line1)
            print("✅ Fixed first min_size issue in output_utils.py")
        
        if old_line2 in content:
            content = content.replace(old_line2, new_line2)
            print("✅ Fixed second min_size issue in output_utils.py")
        
        with open(output_utils_file, 'w') as f:
            f.write(content)
    
    print("�� All ultimate fixes applied! You can now run the training script.")

# Run the ultimate fix
ultimate_fix_yolact_edge()

🔧 Applying ultimate comprehensive fixes to yolact_edge...
✅ Cython already installed
✅ Cython file completely rewritten
⚠️ Import line not found or already fixed
✅ Fixed CUDA issue in yolact.py
⚠️ min_size line not found or already fixed
�� All ultimate fixes applied! You can now run the training script.


In [21]:

# 5. Train Model
'''
!python train.py \
  --config=yolact_edge_mobilenetv2_config \
  --batch_size=8 \
  --dataset=coco2017_dataset \
  --save_folder=weights/ '''

!python train.py --config=yolact_edge_mobilenetv2_config --batch_size=2 --lr=0.001 --save_folder=weights/ --dataset=strawberry_dataset_synthetic


2025-07-13 10:00:45.379459: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-13 10:00:45.390886: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752426045.405652   33519 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752426045.411507   33519 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-13 10:00:45.432229: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:

# 6. Export to ONNX
!python eval.py \
  --trained_model=weights/yolact_edge_mobilenetv2_XX_YY.pth \
  --config=yolact_edge_mobilenetv2_config \
  --export_onnx \
  --onnx_path=weights/yolact_edge.onnx

# 7. Move ONNX to Xavier AGX and Convert to TensorRT:
# trtexec --onnx=yolact_edge.onnx --saveEngine=yolact_edge_fp16.trt --fp16
